In [12]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("adult_train.csv")
test_df = pd.read_csv("adult_test.csv")



### Task 1

Step 1

In [ ]:
y_train = train_df.target
x_train = train_df.drop("target", axis=1)

y_test = test_df.target
x_test = test_df.drop("target", axis=1)